In [1]:
from aocutils.common import ints
from dataclasses import dataclass
from collections import namedtuple
import heapq

Robot = namedtuple('Robot', 'l r u p')
lines=open('day23input.txt').read().splitlines()
maxx = 0
robots =[Robot(*ints(line)) for line in lines]
maxx = max(max(r.r, r.l, r.u) + r.p for r in robots)
size = 2
while size < maxx:
    size *= 2
size

536870912

In [2]:
@dataclass
class RoboBlock():
    left:int
    right:int
    up:int
    leftrange:int
    rightrange:int
    uprange:int
    totalhits:int=0
    
    def split(self):
        midl = self.left + self.leftrange //2 + 1
        midr = self.right + self.rightrange //2 + 1
        midu = self.up + self.uprange // 2 + 1
        for l in (self.left, midl):
            for r in (self.right, midr):
                for u in (self.up, midu):
                    yield RoboBlock(l,r,u,self.leftrange//2, self.rightrange//2, self.uprange//2)
   
    def hits(self, other):
        dis = 0
        dis += max(0, self.left - other.l, other.l - (self.left + self.leftrange))
        dis += max(0, self.right - other.r, other.r - (self.right + self.rightrange))
        dis += max(0, self.up - other.u, other.u - (self.up + self.uprange))
        return dis <= other.p
    
    def count(self, others):
        self.distance = (abs(self.left) + abs(self.right) + abs(self.up))
        self.totalhits = -sum(self.hits(o) for o in others)
    
    def __gt__(self,other):
        return (self.totalhits, self.leftrange, self.distance) > (other.totalhits, other.leftrange, other.distance)

In [3]:
b = RoboBlock(0,0,0,size,size,size)
b.count(robots)
queue = [b]

while queue:
    cur = heapq.heappop(queue)
    if cur.leftrange == 0:
        break
    else:
        for b in cur.split():
            b.count(robots)
            heapq.heappush(queue, b)
cur.distance

121493971

In [9]:
from collections import defaultdict
from z3 import *
nanobots = [ints(line) for line in lines]
def zabs(x):
    return If(x >= 0,x,-x)
def lenr(l):
  return range(len(l))
(x, y, z) = (Int('x'), Int('y'), Int('z'))

in_ranges = [
    Int('in_range_' + str(i)) for i in lenr(nanobots)
]

range_count = Int('sum')
o = Optimize()
for i in lenr(nanobots):
    nx, ny, nz, nrng = nanobots[i]
    o.add(in_ranges[i] == If(zabs(x - nx) + zabs(y - ny) + zabs(z - nz) <= nrng, 1, 0))
o.add(range_count == sum(in_ranges))
dist_from_zero = Int('dist')
o.add(dist_from_zero == zabs(x) + zabs(y) + zabs(z))
h1 = o.maximize(range_count)
h2 = o.minimize(dist_from_zero)
print (o.check())
#print o.lower(h1)
#print o.upper(h1)
print ("b", o.lower(h2), o.upper(h2))
#print o.model()[x]
#print o.model()[y]
#print o.model()[z]

sat
b 121493971 121493971
